## Elastic Net 

In [2]:
import pandas as pd
import numpy as np


In [3]:
data = pd.read_csv("C:\\Users\\user\\Downloads\\Amazon Sale Report.csv\\Amazon Sale Report.csv")

C:\Users\user\AppData\Local\Temp\ipykernel_11160\369179484.py:1: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("C:\\Users\\user\\Downloads\\Amazon Sale Report.csv\\Amazon Sale Report.csv")


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128975 entries, 0 to 128974
Data columns (total 24 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   index               128975 non-null  int64  
 1   Order ID            128975 non-null  object 
 2   Date                128975 non-null  object 
 3   Status              128975 non-null  object 
 4   Fulfilment          128975 non-null  object 
 5   Sales Channel       128975 non-null  object 
 6   ship-service-level  128975 non-null  object 
 7   Style               128975 non-null  object 
 8   SKU                 128975 non-null  object 
 9   Category            128975 non-null  object 
 10  Size                128975 non-null  object 
 11  ASIN                128975 non-null  object 
 12  Courier Status      122103 non-null  object 
 13  Qty                 128975 non-null  int64  
 14  currency            121180 non-null  object 
 15  Amount              121180 non-nul

### Data Cleaning & Transformation

In [5]:

def preprocess_orders(data: pd.DataFrame) -> pd.DataFrame:
    """
    Preprocess the orders dataset:
    - Drop irrelevant or high-cardinality columns
    - Rename columns if needed
    - Extract useful features from 'Date'
    - Create derived features like day, month, weekday, weekend flag
    """
    
    # 1️⃣ Drop unwanted columns if they exist
    DROP_COLS = [
        "index",
        "Order ID",           # pure identifier
        "SKU",                # high cardinality ID
        "promotion-ids",      # sparse / leakage-prone
        "Unnamed: 22",
        "ASIN",
        "fulfilled-by",
        "ship-postal-code",
        "currency",
        "Size"
    ]
    
    data = data.drop(columns=[c for c in DROP_COLS if c in data.columns])
    
    # 2️⃣ Rename columns if necessary
    if 'Sales Channel ' in data.columns:
        data.rename(columns={'Sales Channel ':'Sales Channel'}, inplace=True)
    
    # 3️⃣ Convert 'Date' to datetime safely
    if 'Date' in data.columns:
        data["Date"] = pd.to_datetime(data["Date"], dayfirst=True, errors="coerce")
        
        # Extract date-related features
        data["order_day"] = data["Date"].dt.day_name()
        data["order_month"] = data["Date"].dt.month_name()
        data["order_weekday"] = data["Date"].dt.weekday
        data["is_weekend"] = data["order_weekday"].isin([5,6]).astype(int)
        
        # Drop original Date column
        data = data.drop(columns=["Date"])
    
    return data


In [6]:
data = preprocess_orders(data)
print(data.head())


C:\Users\user\AppData\Local\Temp\ipykernel_11160\12487712.py:34: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data["Date"] = pd.to_datetime(data["Date"], dayfirst=True, errors="coerce")


                         Status Fulfilment Sales Channel ship-service-level  \
0                     Cancelled   Merchant     Amazon.in           Standard   
1  Shipped - Delivered to Buyer   Merchant     Amazon.in           Standard   
2                       Shipped     Amazon     Amazon.in          Expedited   
3                     Cancelled   Merchant     Amazon.in           Standard   
4                       Shipped     Amazon     Amazon.in          Expedited   

     Style       Category Courier Status  Qty  Amount    ship-city  \
0   SET389            Set            NaN    0  647.62       MUMBAI   
1  JNE3781          kurta        Shipped    1  406.00    BENGALURU   
2  JNE3371          kurta        Shipped    1  329.00  NAVI MUMBAI   
3    J0341  Western Dress            NaN    0  753.33   PUDUCHERRY   
4  JNE3671            Top        Shipped    1  574.00      CHENNAI   

    ship-state ship-country    B2B order_day order_month  order_weekday  \
0  MAHARASHTRA           IN  

## Feature Engineering

In [7]:
# seperating numerical and categorical columns
numerical_cols = data.select_dtypes(include="number").columns
print("numerical_cols:\n", numerical_cols)
print()

categorical_cols = data.select_dtypes(exclude="number").columns
print("catgeorical_cols:\n", categorical_cols)

numerical_cols:
 Index(['Qty', 'Amount', 'order_weekday', 'is_weekend'], dtype='object')

catgeorical_cols:
 Index(['Status', 'Fulfilment', 'Sales Channel', 'ship-service-level', 'Style',
       'Category', 'Courier Status', 'ship-city', 'ship-state', 'ship-country',
       'B2B', 'order_day', 'order_month'],
      dtype='object')


In [8]:
numerical_cols = numerical_cols.drop(['Amount'])
numerical_cols

Index(['Qty', 'order_weekday', 'is_weekend'], dtype='object')

In [9]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler


In [10]:
# build a function -> general purpose -> to handle ml preprocessings
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

def build_preprocessor(num_cols, cat_cols, scale_numeric=True):
    
    num_steps = [
        ("imputer", SimpleImputer(strategy="median"))
    ]
    
    if scale_numeric:
        num_steps.append(("scaler", StandardScaler()))
    
    num_pipeline = Pipeline(num_steps)

    cat_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("encoder", OneHotEncoder(handle_unknown="ignore"))
    ])

    return ColumnTransformer([
        ("num", num_pipeline, num_cols),
        ("cat", cat_pipeline, cat_cols)
    ])


In [11]:
# for linear models -> scaling is on!
preprocessor = build_preprocessor(
    num_cols=numerical_cols,
    cat_cols=categorical_cols,
    scale_numeric=True
)

In [12]:

TARGET = "Amount"   # regression target

X = data.drop(columns=[TARGET])
y = data[TARGET]


In [13]:
# fill target with median values
y.fillna(value=data['Amount'].median(), inplace=True)


In [14]:
X.shape

(128975, 16)

In [15]:
y.shape

(128975,)

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)


In [17]:
X_train.shape

(103180, 16)

In [18]:
y_train.shape

(103180,)

In [19]:
def process_and_sample(X_train, X_test, y_train, sample_frac=0.1, n_bins=10, preprocessor=None, random_state=42):
    # 1️⃣ Fit preprocessor on full X_train
    X_train_processed = preprocessor.fit_transform(X_train)
    X_test_processed = preprocessor.transform(X_test)
    
    # 2️⃣ Create quantile bins for stratified sampling
    y_bins = pd.qcut(y_train, q=n_bins, duplicates="drop")
    
    # 3️⃣ Stratified sampling
    X_train_sampled, _, y_train_sampled, _ = train_test_split(
        X_train_processed,
        y_train,
        test_size=1-sample_frac,   # keep fraction for training
        stratify=y_bins,
        random_state=random_state
    )
    
    return X_train_processed, X_test_processed, X_train_sampled, y_train_sampled


In [20]:
X_train_processed, X_test_processed, X_train_sampled, y_train_sampled = process_and_sample(
    X_train, X_test, y_train, sample_frac=0.1, preprocessor=preprocessor
)

print("Full X_train_processed shape:", X_train_processed.shape)
print("Sampled X_train_sampled shape:", X_train_sampled.shape)


Full X_train_processed shape: (103180, 9470)
Sampled X_train_sampled shape: (10318, 9470)


## Model Training

In [21]:
from sklearn.linear_model import ElasticNet, ElasticNetCV
from sklearn.metrics import mean_squared_error, r2_score


In [22]:
# l1_ratio=0.5 → equal mix of L1 and L2
elastic_cv = ElasticNetCV(
    l1_ratio=[0.1, 0.5, 0.9],
    alphas=[0.01, 0.1, 1.0, 10.0],
    cv=5,
    max_iter=5000,
    random_state=42
)

elastic_cv.fit(X_train_sampled, y_train_sampled)


ElasticNetCV(alphas=[0.01, 0.1, 1.0, 10.0], cv=5, l1_ratio=[0.1, 0.5, 0.9],
             max_iter=5000, random_state=42)

In [23]:
print("Best alpha:", elastic_cv.alpha_)
print("Best l1_ratio:", elastic_cv.l1_ratio_)


Best alpha: 0.01
Best l1_ratio: 0.9


In [24]:
y_pred = elastic_cv.predict(X_test_processed)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("R2:", r2)
print("RMSE:", rmse)


R2: 0.6943334665623166
RMSE: 152.13338172333934


In [25]:

feature_names = preprocessor.get_feature_names_out()

coef_df = pd.DataFrame({
    "feature": feature_names,
    "coefficient": elastic_cv.coef_
}).sort_values(by="coefficient", key=abs, ascending=False)

print(coef_df.head(10))


                       feature  coefficient
1232         cat__Style_SET278   448.842429
347           cat__Style_J0285   440.734077
342           cat__Style_J0280   380.851003
136           cat__Style_J0011   335.721310
411           cat__Style_J0379   328.533545
339           cat__Style_J0277   280.785320
16    cat__Fulfilment_Merchant   277.285368
15      cat__Fulfilment_Amazon  -277.194890
1323         cat__Style_SET388   272.525483
343           cat__Style_J0281   268.363791
